# Análisis sobre el impacto del bitcoin en la macroeconomía de El Salvador

In [2]:
import pandas as pd
path_relative = "Datos_bitcoin/Deuda_SV/Deuda_Pública_Total_Mensual.csv"
df_deuda = pd.read_csv(path_relative, encoding='ISO-8859-1', sep=",")

df_deuda = df_deuda.iloc[2:4]
df_deuda

,ï»¿Deuda PÃºblica Total Mensual,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70
2,Concepto,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,...,2024-1(p),2024-2(p),2024-3(p),2024-4(p),2024-5(p),2024-6(p),2024-7(p),2024-8(p),2024-9(p),2024-10(p)
3,Deuda PÃºblica Total,19115.16,19135.14,19286.6,19273.34,19240.87,19280.93,19389.45,20311.53,20273.52,...,20140.36,20217.88,20261.33,20746.49,20792.59,20732.57,20758.87,20859.08,20943.05,20836.88
